In [ ]:
import sqlite3
from flask import Flask, jsonify
def MR_DB():
    connection = sqlite3.connect('MRDining.db')
    cursor = connection.cursor()

    # Restaurants table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Restaurants (
            restaurant_id INTEGER PRIMARY KEY AUTOINCREMENT,
            Name TEXT NOT NULL,
            Location TEXT NOT NULL
        )
    ''')
    # Menu table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Menu (
            Item_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Item_Name TEXT NOT NULL,
            Description TEXT NOT NULL,
            Price REAL NOT NULL,
            restaurant_id INTEGER,
            FOREIGN KEY (restaurant_id) REFERENCES Restaurants(restaurant_id) ON DELETE CASCADE
        )
    ''')
    # orders table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Orders (
            Order_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Student_Name TEXT NOT NULL,
            Price REAL NOT NULL,
            restaurant_id INTEGER,
            FOREIGN KEY (restaurant_id) REFERENCES Restaurants(restaurant_id) ON DELETE CASCADE
        )
    ''')
    # Customization Types (E.g., "Size", "Toppings", "Protein")
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Customizations (
            customization_id INTEGER PRIMARY KEY AUTOINCREMENT,
            Item_ID INTEGER NOT NULL,
            Customization_Type TEXT NOT NULL,  
            FOREIGN KEY (Item_ID) REFERENCES Menu(Item_ID) ON DELETE CASCADE
        )
    ''')

    # Customization Options (Holds Actual Choices)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Customization_Options (
            option_id INTEGER PRIMARY KEY AUTOINCREMENT,
            customization_id INTEGER NOT NULL,
            Option_Name TEXT NOT NULL,  
            FOREIGN KEY (customization_id) REFERENCES Customizations(customization_id) ON DELETE CASCADE
        )
    ''')

     # Order Customizations (Tracks Selected Customizations for Each Order)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Order_Customizations (
            order_customization_id INTEGER PRIMARY KEY AUTOINCREMENT,
            Order_ID INTEGER NOT NULL,
            customization_id INTEGER NOT NULL,
            option_id INTEGER NOT NULL,
            FOREIGN KEY (Order_ID) REFERENCES Orders(Order_ID) ON DELETE CASCADE,
            FOREIGN KEY (customization_id) REFERENCES Customizations(customization_id) ON DELETE CASCADE,
            FOREIGN KEY (option_id) REFERENCES Customization_Options(option_id) ON DELETE CASCADE
        )
    ''')

     # Allergies Table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Allergies (
            allergy_id INTEGER PRIMARY KEY AUTOINCREMENT,
            Allergy_Name TEXT NOT NULL UNIQUE
        )
    ''')

    # Item-Allergy Mapping Table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Item_Allergies (
            item_allergy_id INTEGER PRIMARY KEY AUTOINCREMENT,
            Item_ID INTEGER NOT NULL,
            allergy_id INTEGER NOT NULL,
            FOREIGN KEY (Item_ID) REFERENCES Menu(Item_ID) ON DELETE CASCADE,
            FOREIGN KEY (allergy_id) REFERENCES Allergies(allergy_id) ON DELETE CASCADE
        )
    ''')

    # Check if Restaurants table is empty before inserting
    # Fetchone will be used to retive a single row from the result of a query
    # Running a select statement will return a result set of multiple rows. Fetchone will just grab the next row from that result set. 
    cursor.execute("SELECT COUNT(*) FROM Restaurants")
    if cursor.fetchone()[0] == 0:
        Restaurant_Data = [
            ("Mozzie's Handcrafted Pizza", "Gander Dining Hall"),
            ("Urban Hen", "Gander Dining Hall"),
            ("Sushi by Faith", "Gander Dining Hall"),
            ("Rice It Up", "Gander Dining Hall"),
            ("Epic Eats", "Gander Dining Hall"),
            ("Louie's", "DUC"),
            ("Starbucks", "Library")
    ]
    cursor.executemany("INSERT INTO Restaurants (Name, Location) VALUES (?, ?)", Restaurant_Data)
    connection.commit()

# Mozzies Pizza Restaurant
    # Get Mozzie's ID
cursor.execute("SELECT restaurant_id FROM Restaurants WHERE Name = ?", ("Mozzie's Handcrafted Pizza",))
mozzie_id = cursor.fetchone()

if mozzie_id:
    mozzie_id = mozzie_id[0]

    # Insert menu items if Mozzie's menu is empty
    cursor.execute("SELECT COUNT(*) FROM Menu WHERE restaurant_id = ?", (mozzie_id,))
    if cursor.fetchone()[0] == 0:
        menu_items = [
            ("Build Your Own Pasta", "Customize your own pasta dish with a variety of sauces and toppings.", 11.25, mozzie_id),
            ("Build Your Own Pizza", "Create your perfect pizza with fresh ingredients.", 9.45, mozzie_id),
            ("Build Your Own Gluten-Free Pizza", "A gluten-free option with the same great taste.", 11.45, mozzie_id),
            ("Mac & Cheese", "Classic cheesy goodness made fresh.", 11.25, mozzie_id),
        ]
        cursor.executemany("INSERT INTO Menu (Item_Name, Description, Price, restaurant_id) VALUES (?, ?, ?, ?)", menu_items)
        connection.commit()

    # Get Item ID for Pasta
    cursor.execute("SELECT Item_ID FROM Menu WHERE Item_Name = 'Build Your Own Pasta'")
    pasta_id = cursor.fetchone()

    if pasta_id:
        pasta_id = pasta_id[0]

        # Insert customization types
        cursor.execute("SELECT COUNT(*) FROM Customizations WHERE Item_ID = ?", (pasta_id,))
        if cursor.fetchone()[0] == 0:
            customization_types = ["Portion Size", "Sauce", "Toppings", "Garlic Bread Option", "Pasta Type"]
            for ctype in customization_types:
                cursor.execute("INSERT INTO Customizations (Item_ID, Customization_Type) VALUES (?, ?)", (pasta_id, ctype))
                connection.commit()

        # Fetch customization IDs and types
        cursor.execute("SELECT customization_id, Customization_Type FROM Customizations WHERE Item_ID = ?", (pasta_id,))
        pasta_customizations = cursor.fetchall()

        # Define customization options
        customization_options = {
            "Portion Size": ["Small", "Large"],
            "Sauce": ["No Sauce", "Pesto", "Alfredo Sauce", "Red Sauce", "Rose Sauce"],
            "Toppings": [
                "Grilled Chicken", "Mushrooms", "Spinach", "Extra Cheese", 
                "Artichoke Hearts", "Banana Peppers", "Black Olives", 
                "Chopped Garlic", "No Toppings", "Onions", "Tomatoes", "Pineapple"
            ],
            "Garlic Bread Option": ["None", "Regular Garlic Bread"],
            "Pasta Type": ["Chefs Choice Pasta"]
        }

        # Insert options for each customization type
        for customization_id, ctype in pasta_customizations:
            if ctype in customization_options:
                cursor.executemany(
                    "INSERT INTO Customization_Options (customization_id, Option_Name) VALUES (?, ?)",
                    [(customization_id, option) for option in customization_options[ctype]]
                )
                connection.commit()

        # Insert allergies only if the table is empty
        cursor.execute("SELECT COUNT(*) FROM Allergies")
        if cursor.fetchone()[0] == 0:
            allergies = [
                ("No Allergies",),
                ("Dairy Allergy",),
                ("Egg Allergy",),
                ("Fish Allergy",),
                ("Gluten Allergy",),
                ("Peanut Allergy",),
                ("Sesame Allergy",),
                ("Shellfish Allergy",),
                ("Soy Allergy",),
                ("Tree Nut Allergy",),
                ("Wheat Allergy",)
            ]
            cursor.executemany("INSERT INTO Allergies (Allergy_Name) VALUES (?)", allergies)
            connection.commit()

        # Map all allergies to Pasta (for demo purposes)
        cursor.execute("SELECT allergy_id FROM Allergies")
        allergy_ids = cursor.fetchall()
        cursor.executemany(
            "INSERT INTO Item_Allergies (Item_ID, allergy_id) VALUES (?, ?)",
            [(pasta_id, allergy_id[0]) for allergy_id in allergy_ids]
        )
        connection.commit()

connection.close()


app = Flask(__name__)

@app.route('/menu', methods = ['GET'], endpoint = 'load_menu')
def load_menu():    
    connection = sqlite3.connect('MRDining.db')
    cursor = connection.cursor()
    #Command may change depending on DB Structure
    cursor.execute("SELECT * FROM Menu")
    menu_items = cursor.fetchall()
    return jsonify(menu_items)
    connection.close()

@app.route('/order', methods = ['POST'], endpoint = 'place_order')
def place_order():
    order_data = #TBD, connects to frontend. List Student Name, Price, and Restaurant ID
    connection = sqlite3.connect('MRDining.db')
    cursor = connection.cursor()
    cursor.execute('INSERT INTO Orders (Student_Name, Price, restaurant_id) VALUES (?, ?, ?)', order_data)
    connection.commit()
    connection.close()
    return jsonify({"status": "Order placed successfully"})
